In [1]:
import os, sys
import cv2
import time
import numpy as np
import imutils
from imutils.video import VideoStream
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Running on Python {sys.version}, Tensorflow {tf.__version__}, OpenCV {cv2.__version__}')

Running on Python 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)], Tensorflow 2.5.0, OpenCV 4.5.2


In [2]:
print('Loading models...')
prototxtPath = 'face_detector/deploy.prototxt'
weightsPath = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'
face_detector = cv2.dnn.readNet(prototxtPath, weightsPath)  # Single Shot Multi-box Detector with ResNet-10 (SSD)
face_detector.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
face_detector.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
print('Face detection model loaded. ')
model = load_model('MobileNet mask')
model_img_height, model_img_width = 224, 224  # The dimensions that the model is trained on
print('Face detector and mask detector models loaded.')

Loading models...
Face detection model loaded. 
Face detector and mask detector models loaded.


In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (img_height, img_width) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0,  (300, 300))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    locs, preds = [], []

    for i in range(0, detections.shape[2]):  # loop through each face detected
        # extract the confidence of the detection
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # compute the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([img_width, img_height, img_width, img_height])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(img_width - 1, endX), min(img_height - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel resize it to model dimensions, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.resize(cv2.cvtColor(face, cv2.COLOR_BGR2RGB), (model_img_height, model_img_width))
            face = np.expand_dims(preprocess_input(img_to_array(face)), axis=0)
            locs.append((startX, startY, endX, endY))
            preds.append(tf.nn.softmax(maskNet.predict(face))[0])  # softmax here as model does not have due to using logit for loss
    return locs, preds

In [4]:
stream = cv2.VideoCapture('test/Woman refuses to wear a mask at various locations in Singapore.mp4')
while stream.isOpened():
    ret, frame = stream.read()
    frame = imutils.resize(frame, width=400)  # keep aspect ratio so not using cv2.resize
    locs, preds = detect_and_predict_mask(frame=frame, faceNet=face_detector, maskNet=model)

    for (box, pred) in zip(locs, preds):  # loop over the detected face locations and their corresponding locations
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (noMask, Mask) = pred
        label = "Mask On" if Mask > noMask else "No Mask"
        color = (0, 255, 0) if label == "Mask On" else (0, 0, 255)
        label = f"{label}: {max(Mask, noMask) * 100:.3f}%"  # confidence for mask detector

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)

    # stop when press q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

stream.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-kuwfz3h3\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
